In [ ]:
# Third-party
from astropy.io import fits
from astropy.table import Table
import astropy.coordinates as coord
import astropy.units as u
from astropy.table import Table
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

from pyia import GaiaData

In [ ]:
# orig_g = Table.read('../data/R1500_z_750_cyl_rv.fits', memmap=True)
# mask = np.isfinite(orig_g['radial_velocity']) & ((orig_g['parallax'] / orig_g['parallax_error']) > 6)
# mask.sum()

g = GaiaData('../data/R1500_z_750_cyl_rv.fits')

In [ ]:
c = g.skycoord

In [ ]:
np.isfinite(g.e_bp_min_rp_val).sum(), len(g)

In [ ]:
gal = c.galactic[(g.visibility_periods_used >= 8)]
gal.set_representation_cls('cartesian')

fig, axes = plt.subplots(2, 2, figsize=(8, 8), sharex=True, sharey=True)
axes[0,0].plot(gal.u, gal.v, marker=',', alpha=0.1, linestyle='none')
axes[1,0].plot(gal.u, gal.w, marker=',', alpha=0.1, linestyle='none')
axes[1,1].plot(gal.v, gal.w, marker=',', alpha=0.1, linestyle='none')
axes[0,1].set_visible(False)

axes[0,0].set_xlim(-500, 500)
axes[0,0].set_ylim(-500, 500)

fig.tight_layout()

In [ ]:
plt.hist(g.phot_g_mean_mag[np.isfinite(g.phot_g_mean_mag)], 
         bins=np.linspace(8, 14., 32));
plt.axvline(12.6)

In [ ]:
m = 12.6*u.mag
M = 5.*u.mag
coord.Distance(distmod=m-M)

Note: to be somewhat complete with RVs, we should really only go down to abs. mag ~ 5

Assign reddening by nearest neighbor:

In [ ]:
# idx, sep2d, dist3d = c.match_to_catalog_3d(c, nthneighbor=2)

no_ext = np.isnan(g.e_bp_min_rp_val) | np.isnan(g.a_g_val)
idx, sep2d, dist3d = c[no_ext].match_to_catalog_sky(c, nthneighbor=2)

In [ ]:
e_bprp = g.e_bp_min_rp_val.value
a_g = g.a_g_val.value

e_bprp[no_ext] = e_bprp[idx]
a_g[no_ext] = a_g[idx]

In [ ]:
bprp = (g.phot_bp_mean_mag - g.phot_rp_mean_mag)
G = g.phot_g_mean_mag

bprp0 = bprp - e_bprp*u.mag
G0 = G - a_g*u.mag

In [ ]:
# F5–K5
tmp_fgk_mask = (g.teff_val.value < 6500.) & (g.teff_val.value > 4400)
test_mask = tmp_fgk_mask & (g.distance < 200*u.pc)

In [ ]:
bins = (np.arange(-0.5, 2.5+1e-5, 0.02),
        np.arange(-1, 12+1e-5, 0.02))

fig, axes = plt.subplots(1, 2, figsize=(10, 5), sharex=True, sharey=True)

ax = axes[0]

# H, xe, ye = np.histogram2d(bprp[test_mask], (G - g.distmod)[test_mask], 
H, xe, ye = np.histogram2d(bprp0[test_mask], (G0 - g.distmod)[test_mask], 
                           bins=bins)
ax.pcolormesh(xe, ye, H.T, cmap='Greys',
              norm=mpl.colors.LogNorm(vmin=1E0, vmax=1E4))

nodes = np.array([[-0.05, 1.1],
                  [-0.05, 1.65],
                  [0.55, 3.65],
                  [0.77, 5],
                  [1, 6],
                  [1.2, 6],
                  [1, 5.2],
                  [0.65, 3.3],
                  [0.4, 2.5],
                  [-0.05, 1.1]])
ax.plot(nodes[:, 0], nodes[:, 1], marker='.')
fgk_path = mpl.patches.Path(nodes)
ax.axhline(6, zorder=-100)

ax = axes[1]

# H, xe, ye = np.histogram2d(bprp, G - g.distmod, 
H, xe, ye = np.histogram2d(bprp0, G0 - g.distmod, 
                           bins=bins)
ax.pcolormesh(xe, ye, H.T, cmap='Greys',
              norm=mpl.colors.LogNorm(vmin=1E0, vmax=1E4))
ax.plot(nodes[:, 0], nodes[:, 1], marker='.')

ax.set_ylim(12, -1)
ax.set_xlim(-0.5, 2.5)

fig.tight_layout()

In [ ]:
xy = np.vstack((bprp0, G0 - g.distmod)).T.value

In [ ]:
fgk_mask = fgk_path.contains_points(xy)
fgk_mask.sum()

In [ ]:
gal_fgk = c.galactic[fgk_mask]
gal_fgk.set_representation_cls('cartesian')

fig, axes = plt.subplots(2, 2, figsize=(8, 7.2), sharex=True, sharey=True)
axes[0,0].plot(gal_fgk.u, gal_fgk.v, 
               marker=',', alpha=0.2, linestyle='none')
axes[1,0].plot(gal_fgk.u, gal_fgk.w, 
               marker=',', alpha=0.2, linestyle='none')
axes[1,1].plot(gal_fgk.v, gal_fgk.w, 
               marker=',', alpha=0.2, linestyle='none')
axes[0,1].set_visible(False)

axes[0,0].set_xlim(-500, 500)
axes[0,0].set_ylim(-500, 500)

fig.tight_layout()

In [ ]:
tbl = Table.from_pandas(g[fgk_mask].data)

In [ ]:
tbl['bp_rp0'] = bprp0[fgk_mask]
tbl['G0'] = G0[fgk_mask]

In [ ]:
tbl.write('../data/fgk.fits')